# Visión por Computador - Práctica 2
## Detección de puntos relevantes y Construcción de panoramas

### 1. Detección de puntos Harris multiescala (MOPS, Brown, Szeliski, Winder, 2004). 

#### En este apartado se trata de implementar el detector de regiones descrito en el artículo mencionado. En este algoritmo se suponen que todas las imágenes están tomadas en una escala semejante (zoom y distancia iguales) por lo que el objetivo es extraer todas las regiones relevantes que haya a distintas escalas enteras de las imágenes. Por cada región detectada necesitaremos guardar la siguiente información: (las coordenadas x ,y de su centro, su orientación, su escala)). Usar para ello la estructura que resulte más fácil. Presentar los resultados con las imágenes Yosemite.rar.

##### a) Escribir una función que extraiga la lista potencial de puntos Harris en una imagen a distintas escalas. Para ello construiremos una pirámide Gaussiana de la imagen con 3 escalas usando $\sigma = 1$. Sobre cada nivel de la pirámide usar la función OpenCV cornerEigenValsAndVecs para extraer los mapas de auto-valores de la matriz Harris en cada píxel (( fijar los valores de blockSize en el rango [3-13] y ksize en el rango [3-9]) , usar la versión de nivel de gris de las imágenes). Crear una matriz con el valor del criterio selección Harris asociado a cada píxel usando ($k=0.04$). Implementar la fase de supresión de valores non-máximos sobre dicha matriz. Ordenar de mayor-a-menor en cada escala los puntos resultantes de acuerdo a su valor Harris. Seleccionar al menos 1500 puntos de entre los de mayor valor distribuidos entre las distintas escalas (las escalas más bajas tendrán más puntos: 70-20-10). Mostrar el resultado dibujando un círculo sobre la imagen original de radio proporcional a la escala y centro las coordenadas de los puntos.

##### b) Extraer los valores (x,y, escala) de los puntos resultantes en el apartado anterior en un vector y refinar su posición espacial a nivel sub-píxel usando la función OpenCV usando cornerSubPix()) con la imagen del nivel de pirámide correspondiente. Actualizar los datos (x,y ,escala) de cada uno de los puntos encontrados. 

##### c)  Calcular la orientación relevante de cada punto Harris, usando un alisamiento fuerte de las derivadas en x e y de las imágenes, en la escala correspondiente, como propone el paper MOPS de Brown&Szeliski&Winder. (Apartado 2.5) y añadir la información del ángulo al vector de información del punto. Pintar sobre la imagen de círculos anterior un radio en cada círculo indicando la orientación estimada en ese punto.

- **Apartado a**: para resolver este ejercicio, se han desarrollado varias funciones. Iremos explicándolas poco a poco y viendo su desarrollo.
    
    Antes de realizar la pirámide Gaussiana, lo primero es ver si la imagen es divisible entre el número de divisiones que vamos a hacer. Esto se hace para que, una vez encontrado un punto relevante en la escala $S_i$, sea más sencillo recuperar la posición original en la que se encontraba el píxel, tan solo multiplicando tantas veces como se haya reducido la imagen para encontrar ese punto. De esto se encarga la función ```prepare_img_to_harris_points``` que recibe como parámetro la imagen de la que detectaremos los puntos.
    
    Esta función, recibe la imagen, la pasa a escala de grises y añade tantos píxeles como sean necesarios para poder tener una imagen cuyos puntos Harris sean fáciles de recuperar a la escala original.

In [2]:
def prepare_img_to_harris_points(img):
    # Extraemos las dimensiones de la imagen, para que,
    # en caso de que sean de tamaño impar, añadirle las filas
    # o columnas que correspondan, para que, al reducir,
    # podamos recuperar fácilmente las coordenadas de los puntos
    # harris.
    alt, anch = img.shape[:2]

    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    aux=[]
    # Ensanchamos una fila de la imagen
    if alt % 4 != 0 and anch % 4 == 0:
        aux = np.ones(shape=(alt + alt % 4, anch), dtype=np.uint8)
        insert_img_into_other(img_src=gray_img, img_dest=aux, pixel_left_top_col=0,
                              pixel_left_top_row=0, substitute=True)

    # Ensanchamos una columna de la imagen
    elif alt % 4 == 0 and anch % 4 != 0:
        aux = np.ones(shape=(alt, anch + anch % 4), dtype=np.uint8)
        insert_img_into_other(img_src=gray_img, img_dest=aux, pixel_left_top_col=0,
                              pixel_left_top_row=0, substitute=True)

    # Ensanchamos una fila y una columna de la imagen
    elif alt % 4 != 0 and anch % 4 != 0:
        aux = np.ones(shape=(alt + alt % 4, anch + anch % 4), dtype=np.uint8)
        insert_img_into_other(img_src=gray_img, img_dest=aux, pixel_left_top_col=0,
                              pixel_left_top_row=0, substitute=True)
    # se queda igual que la original
    else:
        aux = np.copy(gray_img)

    return aux


  Lo siguiente, es generar la pirámide Gaussiana con la función ```generate_gaussian_pyramide``` definida en la práctica anterior, y obtener los puntos en los que su valor de Harris supere un umbral mínimo. En este caso, el umbral se ha definido con un mínimo de $10^{-5}$